In [44]:
import numpy as np
import cv2
import tensorflow as tf
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

model = tf.keras.models.load_model("/home/arjun/DM/face.h5")
# EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised","neutral"]
EMOTIONS=["angry",
        "disgust",
        "fear",
        "happy",
        "neutral",
        "sad",
        "surprise"]
# EMOTIONS = ["afraid","angry","disgust","happy","neutral","sad","surprised"]

In [3]:
def prepare(ima):
    IMG_SIZE = 48        # image size
    img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

# Static Test

In [14]:
image=cv2.imread("images.jpeg")
# faces = face_cascade.detectMultiScale(image, 1.3, 5)
# faces = sorted(faces, reverse=True, key = lambda x: (x[2]-x[0]) *(x[3]-x[1]))[0]
# (x,y,w,h)=faces
# roi = image[y-40:y+h+40, x:x+w]
prediction = model.predict([prepare(image)])
preds = prediction[0]
label = EMOTIONS[preds.argmax()]
print(label)
# image = cv2.rectangle(image,(x,y-40),(x+w,y+h+40),(255,0,0),2)
cv2.imshow("image",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

fear


# Live Test

In [46]:
cap=cv2.VideoCapture("/home/arjun/DM/1.mp4")
# cap=cv2.VideoCapture(0)
result = cv2.VideoWriter('1testface.avi',cv2.VideoWriter_fourcc(*'MJPG'), 30, (540, 960)) 
while True:
    ret, img=cap.read()
#     print(img.shape)
    img = cv2.resize(img, (540, 960))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    canvas = np.zeros((256,256,3), dtype="uint8")
    frameclone=img
    try:
        faces = sorted(faces, reverse=True, key = lambda x: (x[2]-x[0]) *(x[3]-x[1]))[0]
        (x,y,w,h)=faces
        img = cv2.rectangle(img,(x,y-20),(x+w,y+h),(255,0,0),2)
        roi = img[y-20:y+h, x:x+w]
        cv2.imshow('img2',roi)
        prediction = (model.predict([prepare(roi)]))
        preds = prediction[0]
        label = EMOTIONS[preds.argmax()]
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            text = "{}: {:.2f}%".format(emotion, prob*100)
            w = int(prob*300)
            cv2.rectangle(canvas, (7, (i*35)+5), (w, (i*35)+35),(0,0,255), -1)
            cv2.putText(canvas, text, (10, (i*35) +23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 2)
            cv2.imshow("Probabilities", canvas)
        
        cv2.putText(img,label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        result.write(img)
    except:
        pass
    cv2.imshow('img',img)
    cv2.waitKey(1)
    if cv2.waitKey(1) & cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.1) /tmp/pip-req-build-ms668fyv/opencv/modules/imgproc/src/resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


# Test on static Validation data

In [16]:

for j in range(0,7):
    right_count=0
    wrong_count=0
    for i in range(1,50):
#         try:
            img=cv2.imread("/home/arjun/DM/Face/validation/"+str(j)+"/"+str(i)+".jpg")
    #         cv2.imshow("image",img)
    #         cv2.waitKey(0)
    #         cv2.destroyAllWindows() 
#             faces = face_cascade.detectMultiScale(img, 1.3, 5)
#             print(faces)
#             faces = sorted(faces, reverse=True, key = lambda x: (x[2]-x[0]) *(x[3]-x[1]))[0]
#             (x,y,w,h)=faces
#             roi = image[y-20:y+h, x:x+w]
            pr=model.predict([prepare(img)])
            preds=pr[0]
            label = EMOTIONS[preds.argmax()]
            if(label==EMOTIONS[j]):
                right_count+=1
            else:
                wrong_count+=1
#         except:
#             pass
    print(EMOTIONS[j])
    print("Right "+str(right_count)+" Wrong "+str(wrong_count))

angry
Right 25 Wrong 24
disgust
Right 33 Wrong 16
fear
Right 18 Wrong 31
happy
Right 36 Wrong 13
neutral
Right 33 Wrong 16
sad
Right 31 Wrong 18
surprise
Right 40 Wrong 9


In [ ]:
angry
Right 20 Wrong 29
disgust
Right 30 Wrong 19
fear
Right 23 Wrong 26
happy
Right 40 Wrong 9
neutral
Right 26 Wrong 23
sad
Right 32 Wrong 17
surprise
Right 34 Wrong 15

